In [5]:
from __future__ import division, print_function
from vpython import *
#create scene
scene = canvas(title='3D scene')

sphere()


<IPython.core.display.Javascript object>

ERROR! Session/line number was not unique in database. History logging moved to new session 35


In [6]:
import numpy as np
from vpython import *

#create scene
scene = canvas(title='3D scene')


sphere1 = sphere(pos = vector(-5,0,0), radius=0.5, color=color.cyan)
sphere2 = sphere(pos=vector(0,-1,0), radius=0.5, color =color.red)
arr = arrow(pos=sphere1.pos, axis = sphere2.pos-sphere1.pos, color=color.yellow)



<IPython.core.display.Javascript object>

In [8]:
import numpy as np
from vpython import *
#create scene
scene = canvas(title='3D scene')


ball = sphere()
box = box(pos=vec(0,-1,0), width= 4, length =4, height=0.5)
trace = curve(radius = 0.2, color = color.green)

for i in range(1000):
    t = i * 0.1
    y = np.sin(t)
    ball.pos = vec(t,y,0)
    trace.append(ball.pos)
    rate(24)


<IPython.core.display.Javascript object>

In [ ]:
from vpython import *
import numpy as np
#create scene
scene = canvas(title='3D scene')


print("""
Click to plot a normalized electric field vector.
Vectors are blue if low magnitude, red if high.
Right button drag or Ctrl-drag to rotate camera to view scene.
Middle button drag or Alt-drag to zoom in or out.
  On a two-button mouse, middle is left + right.
""")

ec = 1.6e-19  # electron charge
Kcoulomb = 8.8988e9 #Nm^2/coul^2
scene.title="Electric Field Vectors"
scene.range = 2e-13

charges = [ sphere( pos = vec (-1e-13,0,0), Q =  ec, color=color.red, radius = 6e-15 ),
            sphere( pos = vec ( 1e-13,0,0), Q = -ec, color=color.blue, radius = 6e-15 ),
          ]

def getfield(p):
    f = vector(0,0,0)
    print (charges)
    for c in charges:
        f = f + (p-c.pos) * 8.988e9 * c.Q / mag(p-c.pos)**3
    return f

while True:
    #p = scene.mouse.getclick().pos
    ev = scene.waitfor('click')
    if ev.event == 'click':
        p = ev.pos

    print(p)
    f = getfield(p)
    m = mag(f)
    red = np.maximum( 1-1e17/m, 0 )
    blue = np.minimum(   1e17/m, 1 )
    if red >= blue:
        blue = blue/red
        red = 1.0
    else:
        red = red/blue
        blue = 1.0
    arrow( pos=p, axis=f * (4e-14/1e17),
            shaftwidth = 6e-15,
            color=vec (red,0,blue))


<IPython.core.display.Javascript object>


Click to plot a normalized electric field vector.
Vectors are blue if low magnitude, red if high.
Right button drag or Ctrl-drag to rotate camera to view scene.
Middle button drag or Alt-drag to zoom in or out.
  On a two-button mouse, middle is left + right.



In [ ]:
from vpython import *
#create scene
scene = canvas(title='3D scene')


N = 3 # N by N by N array of atoms
# Surrounding the N**3 atoms is another layer of invisible fixed-position atoms
# that provide stability to the lattice.
k = 1
m = 1
spacing = 1
atom_radius = 0.3*spacing
L0 = spacing-1.8*atom_radius
V0 = pi*(0.5*atom_radius)**2*L0 # initial volume of spring
scene.center = 0.5*(N-1)*vector(1,1,1)
dt = 0.04*(2*pi*sqrt(m/k))
axes = [vector(1,0,0), vector(0,1,0), vector(0,0,1)]

#scene.caption= """A model of a solid represented as atoms connected by interatomic bonds.
#
#Right button drag or Ctrl-drag to rotate "camera" to view scene.
#To zoom, drag with middle button or Alt/Option depressed, or use scroll wheel.
#  On a two-button mouse, middle is left + right.
#Shift-drag to pan left/right and up/down.
#Touch screen: pinch/extend to zoom, swipe or two-finger rotate."""

class crystal:
        
    def __init__(self,  N, atom_radius, spacing, momentumRange ):
        self.atoms = []
        self.springs = []
        
        # Create (N+2)^3 atoms in a grid; the outermost atoms are fixed and invisible
        for z in range(-1,N+1,1):
            for y in range(-1,N+1,1):
                for x in range(-1,N+1,1):
                    atom = sphere()
                    atom.pos = vector(x,y,z)*spacing
                    atom.radius = atom_radius
                    atom.color = vector(0,0.58,0.69)
                    if 0 <= x < N and 0 <= y < N and 0 <= z < N:
                        p = vec.random()
                        atom.momentum = momentumRange*p
                    else:
                        atom.visible = False
                        atom.momentum = vec(0,0,0)
                    atom.index = len(self.atoms)
                    self.atoms.append( atom )
        for atom in self.atoms:
            if atom.visible:
                if atom.pos.x == 0:
                    self.make_spring(self.atoms[atom.index-1], atom, False)
                    self.make_spring(atom, self.atoms[atom.index+1], True)
                elif atom.pos.x == N-1:
                    self.make_spring(atom, self.atoms[atom.index+1], False)
                else:
                    self.make_spring(atom, self.atoms[atom.index+1], True)

                if atom.pos.y == 0:
                    self.make_spring(self.atoms[atom.index-(N+2)], atom, False)
                    self.make_spring(atom, self.atoms[atom.index+(N+2)], True)
                elif atom.pos.y == N-1:
                    self.make_spring(atom, self.atoms[atom.index+(N+2)], False)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(N+2)], True)
                    
                if atom.pos.z == 0:
                    self.make_spring(self.atoms[atom.index-(N+2)**2], atom, False)
                    self.make_spring(atom, self.atoms[atom.index+(N+2)**2], True)
                elif atom.pos.z == N-1:
                    self.make_spring(atom, self.atoms[atom.index+(N+2)**2], False)
                else:
                    self.make_spring(atom, self.atoms[atom.index+(N+2)**2], True)
    
    # Create a grid of springs linking each atom to the adjacent atoms
    # in each dimension, or to invisible motionless atoms
    def make_spring(self, start, end, visible):
        spring = helix()
        spring.pos = start.pos
        spring.axis = end.pos-start.pos
        spring.visible = visible
        spring.thickness = 0.05
        spring.radius = 0.5*atom_radius
        spring.length = spacing
        spring.start = start
        spring.end = end
        spring.color = color.orange
        self.springs.append(spring)

c = crystal(N, atom_radius, spacing, 0.1*spacing*sqrt(k/m))

while True:
    rate(60)
    for atom in c.atoms:
        if atom.visible:
            atom.pos = atom.pos + atom.momentum/m*dt
    for spring in c.springs:
        spring.axis = spring.end.pos - spring.start.pos
        L = mag(spring.axis)
        spring.axis = spring.axis.norm()
        spring.pos = spring.start.pos+0.5*atom_radius*spring.axis
        Ls = L-atom_radius
        spring.length = Ls
        Fdt = spring.axis * (k*dt * (1-spacing/L))
        if spring.start.visible:
            spring.start.momentum = spring.start.momentum + Fdt
        if spring.end.visible:
            spring.end.momentum = spring.end.momentum - Fdt


ERROR! Session/line number was not unique in

<IPython.core.display.Javascript object>